# Cell Optimization

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation

import ase.io
from ase.lattice.cubic import FaceCenteredCubic
from ase.build import bulk

from apps.calcexamples.structure_browser import StructureBrowser


import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget

import nglview

In [ ]:
atoms = None
node = False
structures = [("Select a Structure",{"status":False})]

layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

viewer = nglview.NGLWidget()
clear_output()

In [ ]:
def refresh_structure_view():
    global viewer, atoms, node
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    if node is False:
        return
    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()

## Step 1: Select Structure

In [ ]:
def on_struct_change(c):
    global atoms, node
    node = struct_browser.results.value
    refresh_structure_view()
    if not node:
        return
    cell_params.value = "Unit cell <br />a = {:.2f} {:.2f} {:.2f} <br />b = {:.2f} {:.2f} {:.2f} <br />c = {:.2f} {:.2f} {:.2f}".format(
        atoms.cell[0][0], atoms.cell[0][1], atoms.cell[0][2],
        atoms.cell[1][0], atoms.cell[1][1], atoms.cell[1][2],
        atoms.cell[2][0], atoms.cell[2][1], atoms.cell[2][2])

    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()
cell_params = ipw.HTML("Cell: ")
clear_output()
display(ipw.VBox([struct_browser, viewer, cell_params]))

## Step 2: Optimize the unit cell

In [ ]:
from aiida.orm import load_node, Code
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.upf import get_pseudos_from_structure
from aiida.orm.utils import CalculationFactory
from aiida.work.run import submit

In [ ]:
def setup_calc():
    options =  {'max_wallclock_seconds': 3600, 'resources': {'num_machines': 1, "num_mpiprocs_per_machine": 1,}}

    codename = code_group.children[1].value['cp2k']
    if codename is None:
        print ("Please select a code")
        return None
    if node is False:
        print ("Please select a structure")
        return None
    code = Code.get_from_string(codename)

    parameters = {
        'GLOBAL': {
            'RUN_TYPE': 'CELL_OPT',
        },
        'FORCE_EVAL': {
            'METHOD': 'Quickstep',
            'STRESS_TENSOR': 'Analytical',
            'DFT': {
                'BASIS_SET_FILE_NAME': 'BASIS_MOLOPT',
                'POTENTIAL_FILE_NAME': 'POTENTIAL',
                'SCF': {
                    'MAX_SCF': 15,
                    'EPS_SCF': 1e-7,
                    'OT': {
                        'PRECONDITIONER': 'FULL_SINGLE_INVERSE',
                        'MINIMIZER': 'CG'
                    },
                    'OUTER_SCF': {
                        'MAX_SCF': 15,
                        'EPS_SCF': 1e-7,
                    },
                },
                'QS': {
                    'EPS_DEFAULT': 1.0e-12,
                    'WF_INTERPOLATION': 'ps',
                    'EXTRAPOLATION_ORDER': 3,
                },
                'MGRID': {
                    'NGRIDS': 4,
                    'CUTOFF': 280,
                    'REL_CUTOFF': 30,
                },
                'XC': {
                    'XC_FUNCTIONAL': {
                        '_': 'PBE',
                    },
                },
            },
            'SUBSYS': {
                'KIND': [{'_': e , 'BASIS_SET': 'DZVP-MOLOPT-SR-GTH','POTENTIAL': 'GTH-PBE'} for e in node.get_kind_names()],
            },
        }
    }

    inputs = {
        'code': code,
        'structure': node,
        'parameters': ParameterData(dict=parameters),
        'settings': ParameterData(dict={}),
        '_options': options,
    }
    return inputs

In [ ]:
def get_code_options(plugin_classes):
    """
    Return AiiDA codes using a specific set of plugins
    
    :param plugin_classes: a dictionary of the type
      {'pw': 'quantumespresso.pw', 'ph': 'quantumespresso.ph'}
      where the key is a label and the value is the plugin to check for.
      It will return the set of codes that exist on the same machine.
    """
    from aiida.orm.querybuilder import QueryBuilder
    from aiida.orm import Code, Computer
    from aiida.backends.utils import get_automatic_user
    
    current_user = get_automatic_user()
    
    qb = QueryBuilder()
    qb.append(Computer,
          filters={'enabled': True},
          project=['*'], tag='computer')
    ordered_tags = []
    for tag, plugin_class in plugin_classes.iteritems():
        ordered_tags.append(tag)
        qb.append(Code,
          filters={'attributes.input_plugin': {'==': plugin_class},
                   'extras.hidden': {"~==": True}
            },
            project='label', tag='{}code'.format(tag), has_computer='computer')
    all_results = qb.all()
    # Filter in python only the ones that are actually user_configured
    # codeset[0] is the computer
    # codeset[1:] are the various code names, depending on the ones asked in input
    return [{tag: "{}@{}".format(codename, codeset[0].name) for codename, tag in zip(codeset[1:], ordered_tags)} 
            for codeset in all_results 
            if codeset[0].is_user_configured(current_user) and codeset[0].is_user_enabled(current_user)]

def get_code_dropdown(classes):
    """
    This function returns a group containing a dropdown list to select a
    valid available Quantum ESPRESSO pw.x code.

    To use it::

      code_group = get_code_pwonly_dropdown()


    You can later retrieve the value as follows::
   
      from IPython.display import display
      code_group = get_code_pwonly_dropdown()
      display(code_group)

    If this is None, then no code was found.
    Otherwise it will be a dictionary, where the only available key
    is 'pw' and the value is the code name, so you can get the code as::

       code_name = code_names['pw']
       code = Code.get_from_string(code_name)
    """
    import ipywidgets as ipw

    code_options_full = None
    in_codename = ipw.Dropdown(options=[], disabled=True)

    code_options_full = get_code_options(plugin_classes=classes)
    code_strings = ["{}".format(code_option['cp2k']) 
        for code_option in code_options_full]  
        
    if code_options_full is None:
        in_codename.options=[["Error while retrieving the list of codes", None]]
        in_codename.disabled=True
        in_codename.value = None
    elif not code_options_full:
        in_codename.options = [["No AiiDA codes configured yet", None]]
        in_codename.disabled = True
        in_codename.value = None
    else:
        code_options = zip(code_strings, code_options_full)
        in_codename.options=code_options
        in_codename.disabled = False
        # Set default value (first entry)
        in_codename.value = code_options[0][1]    
                
    code_group = ipw.HBox(
        [
            ipw.Label(value="Select a quantum code to use: "), 
            in_codename,
        ])

    return code_group

code_group = get_code_dropdown(classes={'cp2k': 'cp2k'})

In [ ]:
Cp2kCalculation = CalculationFactory('cp2k')

In [ ]:
def on_click_submit(b):
    with submit_out:
        process = Cp2kCalculation.process()
        inputs = setup_calc()
        if inputs is None:
            pass
        else:
            calculation = submit(process, **inputs)
            print("Submitted calculation N {}".format(calculation.pid))
            print("Once the calculation will be completed the results will be automatically put in the database")

btn_submit = ipw.Button(description='Submit Cell Opt')
btn_submit.on_click(on_click_submit)
submit_out = ipw.Output()
display(ipw.HBox([code_group,btn_submit]), submit_out)